In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
import numpy as np
import pickle
from state_approximation import entanglement_entropy, mpo2mps
from misc import mps_overlap
from disentanglers import disentangle_S2

In [70]:
with open("single_split_state.pkl", "rb") as f:
    state = pickle.load(f)
state = mpo2mps(state)

In [71]:
entanglement_entropy(state)

0.24232390144270416

In [73]:
def _sweep_disentangle(Psi):
    L = len(Psi)
    for i in range(L-1):
        theta = np.tensordot(Psi[i], Psi[i+1], [2,1]).transpose([1,0,2,3])
        theta, U = disentangle_S2(theta, max_iter=200, eps=1.e-16)
        chiL, d1, d2, chiR = theta.shape
        q, r = np.linalg.qr(theta.reshape(chiL*d1, chiR*d2))
        Psi[i] = q.reshape((chiL, d1, -1)).transpose([1,0,2])
        Psi[i+1] = r.reshape((-1, d2, chiR)).transpose([1,0,2])
    return Psi

def sweep_disentangle(Psi):
    Psi = [psi.transpose([0,2,1]) for psi in Psi[::-1]]
    Psi = _sweep_disentangle(Psi)
    Psi = [psi.transpose([0,2,1]) for psi in Psi[::-1]]
    Psi = _sweep_disentangle(Psi)
    return Psi

In [74]:
for i in range(100):
    print(entanglement_entropy(state))
    state = sweep_disentangle(state)
    print(entanglement_entropy(state))

0.000451338201465304
0.000421624821510616
0.000421624821510616
0.0003967830455234092
0.0003967830455234092
0.0003641550616976253
0.0003641550616976253
0.0003251209983043786
0.0003251209983043786
0.00029748799741574665
0.00029748799741574665
0.0002781582781965547
0.0002781582781965547
0.00025673930622940086
0.00025673930622940086
0.00024044067960887717
0.00024044067960887717
0.0002193650954589586
0.0002193650954589586
0.00020362688738101366
0.00020362688738101366
0.00018957418132396525
0.00018957418132396525
0.00017121916360026088
0.00017121916360026088
0.0001575212136822364
0.0001575212136822364
0.000142558319465506
0.000142558319465506
0.0001357172230401527
0.0001357172230401527
0.00012465125369247958
0.00012465125369247958
0.00011611311041761923
0.00011611311041761923
0.00010899921313478028
0.00010899921313478028
0.0001004038094142501
0.0001004038094142501
9.284771155573006e-05
9.284771155573006e-05
8.724860985762171e-05
8.724860985762171e-05
7.980788360752178e-05
7.980788360752178e-